In [ ]:
import torch
from torch import nn
import numpy as np
import gymnasium as gym
from torch.distributions.categorical import Categorical
from torch.optim import Adam

In [ ]:
class Policy(nn.Module):
    def __init__(self,input_size, output_size):
        super(Policy,self).__init__()
        module = [
            nn.Linear(input_size,64),
            nn.ReLU(inplace=True),
            nn.Linear(64,output_size),
            nn.Softmax(output_size)
        ]
        self.mlp = nn.Sequential(*module)
    
    def forward(self,state):
        return self.mlp(state)

class Value(nn.Module):
    def __init__(self, input_size, output_size):
        super(Value,self).__init__()
        module = [
            nn.Linear(input_size,64),
            nn.ReLU(inplace=True),
            nn.Linear(64,output_size)
        ]
        self.mlp = nn.Sequential(*module)

    def forward(self,state):
        return self.mlp(state)    

def train(env_name='CartPole-v0', lr=1e-2, batch_size=5000, epochs=50, render=False):
    env = gym.make(env_name)
    obs_size = env.observation_space.shape[0]
    action_size = env.action_space.shape[0]

    policy = Policy(obs_size,action_size)
    value = Value(obs_size,1)    
    
    value_optimizer = Adam(value.parameters(), lr =1e-3)

    def get_policy(state):
        return Categorical(logits=policy(state))
    
    def get_action(state):
        return get_policy(state).sample().item()

    def advantage_function(state):
        return value(state)

    def compute_loss_advantage(state,weights):
        return torch.nn.functional.mse_loss(advantage_function(state)-weights)

    def estimate_policy_gradient(s)

    def surrogate_advantage(old_policy, new_policy, advantage):
        return NotImplementedError
    
    def conjugate_gradient():
        return NotImplementedError
    
    def reward_to_go(ep_rews):
        rtg = np.zeros(len(ep_rews))
        for i in reversed(range(len(ep_rews))):
            rtg [i] = ep_rews[i] + (rtg[i+1] if i+1 < len(ep_rews) else 0)
        return rtg
    
    
    def train_epoch():
        batch_act = []
        batch_obs= []
        batch_return = []
        batch_len = []
        batch_weight = []
        ep_reward = []

        obs = env.reset()
        done = False 

        while True: 
            batch_obs.append(obs.copy())
            act = get_action(obs)
            obs, reward, done, _ = env.step(act)
            batch_act.append(act)
            ep_reward.append(reward)
            
            if done: 
                total_reward, ep_len = sum(ep_reward), len(ep_reward)
                batch_return.append(total_reward)
                batch_len.append(ep_len)

                batch_weight += list(reward_to_go(ep_reward))
                
                if len(batch_obs > batch_size):
                    break 

        


        loss_value = compute_loss_advantage(torch.is_tensor(batch_obs), torch.is_tensor(batch_weight)) 
        loss_value.backward()
        value_optimizer.step()         



